In [3]:
import menpo3d.io as m3io
import menpo.io as mio
import itertools
from menpo.transform import Rotation
import numpy as np
# from kfdata import load_kf
# from menpodetect import load_dlib_frontal_face_detector
from menpofit.aam import load_balanced_frontal_face_fitter
from menpo.shape import TriMesh, ColouredTriMesh, TexturedTriMesh
from menpo3d.rasterize import rasterize_mesh_from_barycentric_coordinate_images, rasterize_barycentric_coordinate_images
from menpo3d.camera import PerspectiveProjection, PerspectiveCamera
# from mesh_clean import *
from menpo3d.correspond import *
# from KF_landmark_annotation import *
from menpo.shape import PointCloud
from menpo.landmark.labels import face_ibug_68_to_face_ibug_49

In [1]:
import menpo3d.io as m3io
import menpo.io as mio

In [11]:
# y=mio.import_pickle('/home/thanos/Dropbox/pickle_files/template.pkl')
# mesh_land = m3io.import_landmark_file('/home/thanos/Dropbox/pickle_files/ibug68.ljson')
y= m3io.import_mesh('/home/thanos/Downloads/base.obj')
mesh_land = m3io.import_landmark_file('/home/thanos/Downloads/base.obj_ibug68.ljson') 
y.landmarks['ibug49'] = face_ibug_68_to_face_ibug_49(mesh_land)

In [5]:
print(type(mesh_land))

<class 'menpo.shape.labelled.LabelledPointUndirectedGraph'>


In [6]:
print(y.landmarks['ibug49'])

LabelledPointUndirectedGraph: n_labels: 6, n_points: 49, n_edges: 47


In [30]:
%matplotlib qt
y.view(colour='g')
PointCloud(mesh_land.points).view(new_figure=False,marker_face_colour_lms='b',marker_size_lms=120)

In [26]:
PointCloud(mesh_land.points)

PointCloud: n_points: 68, n_dims: 3

In [ ]:
mesh = m3io.import_mesh('/home/stylianos/Dropbox/landmarker/meshes/merge3d.obj')
mesh_land = m3io.import_landmark_file('/home/stylianos/Dropbox/landmarker/meshes/landmarks/merge3d.obj_ibug68.ljson')
mesh.landmarks['ibug49'] = face_ibug_68_to_face_ibug_49(mesh_land)

In [3]:
%matplotlib qt
mesh.view()
PointCloud(mesh_land.points).view(new_figure=False)

In [4]:
shape_mean = m3io.import_mesh('/home/stylianos/Dropbox/landmarker/meshes/base.obj')
model_landmarks_49 =face_ibug_68_to_face_ibug_49(m3io.import_landmark_file('/home/stylianos/Dropbox/landmarker/meshes/landmarks/base.obj_ibug68.ljson'))
shape_mean.landmarks['ibug49'] = model_landmarks_49

In [5]:
#Start the alignment with the basel and Neanderthal scan
from menpo.transform import AlignmentSimilarity
alignment = AlignmentSimilarity(mesh.landmarks['ibug49'], model_landmarks_49)

aligned_mesh = alignment.apply(mesh)
aligned_mesh.landmarks['ibug49'] = alignment.apply(mesh.landmarks['ibug49'])

shape_mean.view()
aligned_mesh.view(new_figure=False,colour=(0,1,0.5))

aligned_mesh.view()
PointCloud(aligned_mesh.landmarks['ibug49'].points).view(new_figure=False,colour=(0,1,0.5),marker_size=0.05)

In [79]:
#Perform NICP to get the shape from the Kinect scan
#stiffness_values=[30, 20,10,2, 1, 0.5, 0.1]
lm_weight=[0.2, 0.2, 0.1, 0.1, 0.1, 0, 0 ,0]
data = [2, 2, 0.8, 0.8, 0.4, 0, 0 ,0]
results = non_rigid_icp(shape_mean, aligned_mesh,verbose=True,generate_instances=True,
                        landmark_group='ibug49',stiffness_weights=None,landmark_weights=lm_weight)
results = list(results)

'ibug49' landmarks will be used as a landmark constraint.
performing similarity alignment using landmarks
using default stiffness_weights: [50, 20, 5, 2, 0.8, 0.5, 0.35, 0.2]
using user defined landmark_weights: [0.2, 0.2, 0.1, 0.1, 0.1, 0, 0, 0]
Not customising data_weights
1/8: stiffness: 50  lm_weight: 0.2
 - 1 stop crit: 0.512  total: 15%  norms: 15%  edges: 0%  lm_err: 0.0160
 - 2 stop crit: 0.019  total: 15%  norms: 15%  edges: 0%  lm_err: 0.0154
 - 3 stop crit: 0.003  total: 16%  norms: 16%  edges: 0%  lm_err: 0.0157
 - 4 stop crit: 0.002  total: 16%  norms: 16%  edges: 0%  lm_err: 0.0162
 - 5 stop crit: 0.001  total: 16%  norms: 16%  edges: 0%  lm_err: 0.0166
2/8: stiffness: 20  lm_weight: 0.2
 - 1 stop crit: 0.001  total: 16%  norms: 16%  edges: 0%  lm_err: 0.0164
 - 2 stop crit: 0.001  total: 15%  norms: 15%  edges: 0%  lm_err: 0.0161
3/8: stiffness: 5  lm_weight: 0.1
 - 1 stop crit: 0.004  total: 15%  norms: 15%  edges: 0%  lm_err: 0.0146
 - 2 stop crit: 0.003  total: 12%  n

In [80]:
#View the Results of the NICP-Kinect Scan
results[-1][0].view()
aligned_mesh.view(new_figure=False, colour=(0,1,0.5))
results[-1][0].view()
nicp_mesh = results[-1][0]

# KD tree search for colouring

In [37]:
from scipy.spatial import KDTree
from menpo3d.vtkutils import trimesh_to_vtk, VTKClosestPointLocator

In [38]:
def duplicate_vertices(mesh):
    # generate a new mesh with unique vertices per triangle
    # (i.e. duplicate verts so that each triangle is unique)    old_to_new = mesh.trilist.ravel()
    old_to_new = mesh.trilist.ravel()
    new_trilist = np.arange(old_to_new.shape[0]).reshape([-1, 3])
    new_points = mesh.points[old_to_new]
    return TriMesh(new_points, trilist=new_trilist), old_to_new

In [41]:
a1 = duplicate_vertices(aligned_mesh)[0]
a2 = duplicate_vertices(nicp_mesh)[0]

In [47]:
kdtree = KDTree(aligned_mesh.points)

In [51]:
a = kdtree.query(nicp_mesh.points, k=3)

In [52]:
from menpo.transform import image_coords_to_tcoords
from menpo.shape import TexturedTriMesh

# tcoords_miss = image_coords_to_tcoords(miss_img.shape).apply(aligned_miss.with_dims([0, 1])).points

T_mesh = TexturedTriMesh(nicp_mesh.points, aligned_mesh.tcoords.points[a[1][:, 2]], aligned_mesh.texture,nicp_mesh.trilist)
T_mesh.view()

# Coloured Trimesh

In [82]:
from menpo3d.barycentric import barycentric_coordinates_of_pointcloud
bc, i = aligned_mesh.barycentric_coordinates_of_pointcloud(PointCloud(nicp_mesh.points))
colour = aligned_mesh.sample_texture_with_barycentric_coordinates(bc,i)

col_nicp = ColouredTriMesh(points=nicp_mesh.points,trilist=nicp_mesh.trilist,colours=colour)
col_nicp.view()

# Textured Trimesh

In [104]:
aligned_mesh.tcoords

PointCloud: n_points: 120084, n_dims: 2

In [102]:
aligned_mesh.n_points

120084

In [92]:
TexturedTriMesh(nicp_mesh.points,aligned_mesh.tcoords,aligned_mesh.texture, nicp_mesh.trilist).view()

TypeError: Couldn't translate array's type to VTK